# Optimizacija fazi kontrolera za vožnju automobila korišćenjem genetskog algoritma

## Uvod

Cilj ovog rada je bila optimizacija fazi kontrolera za vožnju korišćenjem genetskog algoritma. Kontroler je upravljao automobilom koji se kretao po otvorenom ili zatvorenom putu, u zavisnosti od zadatih opcija.  

U radu je prvo pokazano da je moguće konstruisati fazi kontroler koji će pravilno upravljati automobilom, a zatim je pokazano i da je moguće optimizovati nasumično izgenerisan fazi sistem uz pomoć genetskog algoritma tako da pravilno upravlja automobilom.  

Automobilom se upravlja pravilno ako se automobil ne sudara sa ivicama puta i ako se kreće po sredini puta, koliko god je to moguće.

Automobil je testiran na dve vrste puta. Prvi tip puta je krivudav put koji je izgenerisan sinusiodom, a drugi tip puta je bio zatvoren kružni put izgenerisan pomoću konveksnog omotača nasumično odabranog skupa koordinata.

## Fazi sistem

Fazi sistem se sastoji iz 3 ulaza i 2 izlaza. Ulazi su predstavljali udaljenost od kraja puta, a izlazi su predstavljali promenu ugla i brzine.

### Ulaz

Postoje 3 ulazne funkcije:
- `left_sensor`
- `right_sensor`
- `front_sensor`

Svaka od njih ima 4 iste funkcije pripadnosti:  
- `close`
- `midrange`
- `far`
- `very far`

One označavaju koliko smo udaljenosti od svake od ivica puta. 

### Izlaz

Postoje 2 izlazne funkcije:
- `velocity`
- `angle`

`velocity` označava promenu brzine, dok `angle` označava promenu ugla skretanja volana.

`velocity` ima 4 funkcije pripadnosti:
- `low`
- `middle`
- `high`
- `very high`

`angle` ima 5 funkcija pripadnosti:
- `hard right`
- `right`
- `forward`
- `left`
- `hard left`

Sve funkcije pripadnosti su trapeznog oblika, sem funkcija na kraju i na početku intervala, čiji je oblik najsličniji ramp funkciji. Takođe, sve funkcije su imaju interval iz kog smeju da uzimaju vrednosti. Funkcije se mogu lepo videti u sledećem rečniku, koji se koristi pri generisanju nasumičnog fazi sistema:


In [ ]:
ALL_FUZZY_FUNCS = {
    "left_sensor": {
        "name": "left_sensor",
        "left_boundary": 0,
        "right_boundary": 50,
        "mf_names": ["close", "midrange", "far", "very_far"],
        "is_input": True,
    },
    "right_sensor": {
        "name": "right_sensor",
        "left_boundary": 0,
        "right_boundary": 50,
        "mf_names": ["close", "midrange", "far", "very_far"],
        "is_input": True,
    },
    "front_sensor": {
        "name": "front_sensor",
        "left_boundary": 0,
        "right_boundary": 20,
        "mf_names": ["close", "midrange", "far", "very far"],
        "is_input": True,
    },
    "velocity": {
        "name": "velocity",
        "left_boundary": 1,
        "right_boundary": 20,
        "mf_names": ["low", "middle", "high", "very high"],
        "is_input": False,
    },
    "angle": {
        "name": "angle",
        "left_boundary": -45,
        "right_boundary": 45,
        "mf_names": ["hard right", "right", "forward", "left", "hard left"],
        "is_input": False,
    }
}

## Fazi sistem koji upravlja automobilom

U ovom poglavlju je opisan fazi sistem koji je napravljen bez upotrebe genetskog algoritma, da bi bilo pokazano da samo fazi logika može upravljati automobilom, a i da bi postojala nekakva referenca za rezultat genetskog algoritma.

Inicijalizacije ovog fazi sistema se nalaze u fajlu `fuzzy_default.py`:

In [ ]:
import fuzzy

def generate_default_system():
    left = {
        "close"    : MFInput("close", np.array([0, 16]), np.array([1, 0])),
        "midrange" : MFInput("midrange", np.array([10, 14, 18, 24]), np.array([0, 1, 1, 0])),
        "far"      : MFInput("far", np.array([20, 36, 44]), np.array([0, 1, 0])),
        "very far" : MFInput("very far", np.array([32, 100]), np.array([0, 1]))
    }

    right = {
        "close"    : MFInput("close", np.array([0, 16]), np.array([1, 0])),
        "midrange" : MFInput("midrange", np.array([10, 16, 20, 28]), np.array([0, 1, 1, 0])),
        "far"      : MFInput("far", np.array([24, 40, 45]), np.array([0, 1, 0])),
        "very far" : MFInput("very far", np.array([35, 100]), np.array([0, 1]))
    }

    front = {
        "close"    : MFInput("close", np.array([0, 18]), np.array([1, 0])),
        "midrange" : MFInput("midrange", np.array([14, 20, 24, 28]), np.array([0, 1, 1, 0])),
        "far"      : MFInput("far", np.array([26, 34, 40]), np.array([0, 1, 0])),
        "very far" : MFInput("very far", np.array([35, 100]), np.array([0, 1])),
    }

    velocity = {
        "low"       : MFOutput("low", np.array([1, 2]), np.array([1, 0])),
        "middle"    : MFOutput("middle", np.array([1.5, 3, 3.5, 4]), np.array([0, 1, 1, 0])),
        "high"      : MFOutput("high", np.array([3.5, 4.5, 5.5]), np.array([0, 1, 0])),
        "very high" : MFOutput("very high", np.array([4, 8]), np.array([0, 1]))
    }

    angle = {
        "hard right" : MFOutput("hard right", np.array([25, 45]), np.array([1, 0])),
        "right"      : MFOutput("right", np.array([5, 22, 25]), np.array([0, 1, 0])),
        "forward"    : MFOutput("forward", np.array([-5, 0, 5]), np.array([0, 1, 0])),
        "left"       : MFOutput("left", np.array([-25, -22, -5]), np.array([0, 1, 0])),
        "hard left"  : MFOutput("hard left", np.array([-45, -25]), np.array([0, 1]))
    }

    left_sensor = FuzzyInput("left_sensor", np.array([
        left["close"],
        left["midrange"],
        left["far"],
        left["very far"]
    ]))

    right_sensor = FuzzyInput("right_sensor", np.array([
        right["close"],
        right["midrange"],
        right["far"],
        right["very far"]
    ]))

    front_sensor = FuzzyInput("front_sensor", np.array([
        front["close"],
        front["midrange"],
        front["far"],
        front["very far"] 
    ]))

    f_velocity = FuzzyOutput("velocity", np.array([
        velocity["low"],
        velocity["middle"],
        velocity["high"],
        velocity["very high"]
    ]))

    f_angle = FuzzyOutput("angle", np.array([
        angle["hard right"],
        angle["right"],
        angle["forward"],
        angle["left"],
        angle["hard left"]
    ]))

    return left_sensor, right_sensor, front_sensor, f_velocity, f_angle 

## Izgled Fazi klase

O ovom poglavlju je opisana generalna struktura fuzzy.py fajla

##  Opis genetic_algorithm

## Opis fitness funkcije

## Rezultati

## Zaključak